<a href="https://colab.research.google.com/github/fact-h/kaggle-sklearn-ML-tutorial/blob/main/Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipelines

## 介绍
pipelines将数据预处理和建模过程统一组织起来，作为一个整体，使用pipeline可以一步完成数据预处理和建模。好处有：
1. 使代码更简洁
2. 不易出错
3. 容易部署
4. 有更多的模型验证方法，比如交叉验证


In [ ]:
#@title 得到X_train, X_valid, y_train, y_valid
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

### 第一步：定义预处理步
使用`ColumnTransformer`类将不同预处理步骤合在一起。

- 在**数值**数据中插入缺少值
- 在**分类**数据中插入缺少值并应用`one-hot`编码

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data 对数值数据进行预处理-对缺少值插补
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data 对分类数据进行预处理-插补和onehot
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
# 将数值数据和分类数据绑定在一起
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## 第二步：定义模型

使用`RandomForestRegressor`类定义随机森林模型

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

## 第三步：创建和评估Pipeline
使用`Pipeline`类定义pipeline，将数据预处理和建模步骤绑定在一起。
- 使用 pipeline，用一行代码处理训练数据并拟合模型
- 使用 pipeline，可直接用未处理的特征`X_valid`做预测，pipeline 会在生成预测结果前自动对特征进行预处理

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)